In [17]:
import pyodbc
import pandas as pd
import math

con = pyodbc.connect('DRIVER={SQL Server};SERVER=localhost;DATABASE=TASMC_ICU_Intensix')



In [18]:
cursor = con.cursor()


In [19]:
patients = pd.read_csv('ML_patients.txt')

In [20]:
for p in range(len(patients.index)):
    sql = "SELECT [PatientID],[Time] \
    ,MAX( CASE WHEN [ParameterID] in ('4704', \
                                       '6636', \
                                       '6637', \
                                       '7719', \
                                       '8417', \
                                       '8719', \
                                       '8720', \
                                       '9630', \
                                       '10997') THEN [Value] ELSE NULL END) HR \
    ,MAX( CASE WHEN [ParameterID] in ('11069', \
                                      '11086', \
                                      '11070', \
                                      '6687', \
                                      '6688', \
                                      '6695', \
                                      '6696', \
                                      '7165') THEN [Value]  ELSE NULL END) Pulse \
    ,MAX( CASE WHEN [ParameterID] in ('4727', \
                                      '6627', \
                                      '6628', \
                                      '9228', \
                                      '9229', \
                                      '9231', \
                                      '9433', \
                                      '9434', \
                                      '9474') THEN [Value]  ELSE NULL END) ArtBPS \
    ,MAX( CASE WHEN [ParameterID] in ('5433', \
                                      '9472', \
                                      '10825', \
                                      '10999', \
                                      '11084') THEN [Value]  ELSE NULL END) ArtBPM \
    ,MAX( CASE WHEN [ParameterID] in ('4721', \
                                      '5532', \
                                      '5644', \
                                      '5945', \
                                      '5952', \
                                      '6380', \
                                      '7104') THEN [Value]  ELSE NULL END) CVP \
    ,MAX( CASE WHEN [ParameterID] in ('11079') THEN [Value] ELSE NULL END) PPV \
    ,MAX( CASE WHEN [ParameterID] in ('9014') THEN [Value] ELSE NULL END) SPV \
    ,MAX( CASE WHEN [ParameterID] in ('7170') THEN [Value] ELSE NULL END) SVV \
    ,MAX( CASE WHEN [ParameterID] in ('6173','10826') THEN [Value] ELSE NULL END)  NiBPM \
    ,MAX( CASE WHEN [ParameterID] in ('4717','9649') THEN [Value] ELSE NULL END) PAPD \
    ,MAX( CASE WHEN [ParameterID] in ('8364', \
                                      '8365', \
                                      '8605', \
                                      '8606', \
                                      '8607') THEN [Value] ELSE NULL END) BIS \
    ,MAX( CASE WHEN [ParameterID] in ('4612','11017') THEN [Value]  ELSE NULL END ) RR_total \
    ,MAX( CASE WHEN [ParameterID] in ('7295','11016') THEN [Value]  ELSE NULL END ) RR_mandatory \
    ,MAX( CASE WHEN [ParameterID] in ('4631') THEN [Value]  ELSE NULL END ) Spo2 \
    ,MAX( CASE WHEN [ParameterID] in ('4960') THEN [Value]  ELSE NULL END ) ST1 \
    ,MAX( CASE WHEN [ParameterID] in ('4961') THEN [Value]  ELSE NULL END ) ST2 \
    ,MAX( CASE WHEN [ParameterID] in ('4962') THEN [Value]  ELSE NULL END ) ST3 \
    ,MAX( CASE WHEN [ParameterID] in ('4611') THEN [Value]  ELSE NULL END ) Fio2 \
    ,MAX( CASE WHEN [ParameterID] in ('7254') THEN [Value]  ELSE NULL END ) Temperature \
    ,MAX( CASE WHEN [ParameterID] in ('5596') THEN [Value]  ELSE NULL END ) Haemaglobin \
    ,MAX( CASE WHEN [ParameterID] in ('7467') THEN [Value]  ELSE NULL END ) Platelets \
    ,MAX( CASE WHEN [ParameterID] in ('6478') THEN [Value]  ELSE NULL END ) Urea \
    ,MAX( CASE WHEN [ParameterID] in ('7465') THEN [Value]  ELSE NULL END ) WBC \
    ,MAX( CASE WHEN [ParameterID] in ('6472') THEN [Value]  ELSE NULL END ) Neutrophils \
    ,MAX( CASE WHEN [ParameterID] in ('10070') THEN [Value]  ELSE NULL END ) Signal_strength \
    ,MAX( CASE WHEN [ParameterID] in ('6469') THEN [Value]  ELSE NULL END ) Lymphocytes \
    ,MAX( CASE WHEN [ParameterID] in ('6458') THEN [Value]  ELSE NULL END ) Eosinphils \
    ,MAX( CASE WHEN [ParameterID] in ('7399') THEN [Value]  ELSE NULL END ) INR \
    ,MAX( CASE WHEN [ParameterID] in ('4620') THEN [Value]  ELSE NULL END ) PaO2 \
    ,MAX( CASE WHEN [ParameterID] in ('5648') THEN [Value]  ELSE NULL END ) Weight_Actual_kg \
    ,MAX( CASE WHEN [ParameterID] in ('6531') THEN [Value]  ELSE NULL END ) Total_Cholesterol \
    FROM  [TASMC_ICU_Intensix].[dbo].[Signals] \
    WHERE  [TASMC_ICU_Intensix].[dbo].[Signals].[PatientID] = %s\
    GROUP BY [PatientID], [Time] \
    ORDER BY [Time];" %patients.ix[p,'PatientID']
    
    df=pd.read_sql(sql, con)
    
    for i in range(len(df.index)):
        param_type=''
        for j in range(2,len(df.columns)):
            if df.ix[i,j] is not None:
                if not math.isnan(df.ix[i,j]):
                        param_type=param_type + ' ' + df.columns[j]
        cursor.execute("UPDATE [TASMC_ICU_Intensix].[dbo].[Signals] SET Param_Type = ? WHERE [TASMC_ICU_Intensix].[dbo].[Signals].[Time] = ? AND PatientID = ?",(param_type,df.ix[i,'Time'],int(df.ix[i,'PatientID'])));
        con.commit()
    

ProgrammingError: ('42000', "[42000] [Microsoft][ODBC SQL Server Driver][SQL Server]Could not allocate space for object 'dbo.Signals'.'ParamTypeInd' in database 'TASMC_ICU_Intensix' because the 'PRIMARY' filegroup is full. Create disk space by deleting unneeded files, dropping objects in the filegroup, adding additional files to the filegroup, or setting autogrowth on for existing files in the filegroup. (1105) (SQLExecDirectW)")

20174

In [56]:
dtest=df[(df.ArtBPM<50) & (df.CVP<5) & (df.CVP>-10)]


In [379]:
for i in range(len(df.index)):
    t1=df.ix[i-2,'Time'].to_pydatetime()
    t2=df.ix[i-1,'Time'].to_pydatetime()
    t3=df.ix[i,'Time'].to_pydatetime()
    delta1 = t2-t1
    delta2 = t3-t2
    if (t2-t1).total_seconds()==60.0 and (t3-t2).total_seconds()==60.0:

SyntaxError: unexpected EOF while parsing (<ipython-input-379-8cf74795cc88>, line 7)